*About this notebook:* prepared for Ahead of the Storm Hackathon, UN, Open Source Week, 16-17 June 2025

**Challenge 1: 
Unlocking Child-Centric Extreme Weather Intelligence: 
From Hindcasting to Forecasting, from Reaction to Proaction**

Prepared by team **SAFE (Storm Action For EveryChild)**:

Ngoc Tram Nguyen, Orsolya Horváth-Dudás, Rita Mateus, Shruthiveena Krishnamurthy,  Varvara Krechetova


*Case Study: Bangladesh*

The notebooks describes workflow, data and functions for identifying level of risks by risk domain and calculating number of affected children in given areas (flood risk zones, hurricane impact area, etc.)


In [1]:
import os
import rasterio
import numpy as np
from rasterio.features import shapes
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape, Point
from rasterio.crs import CRS
from rasterio.mask import mask
from rasterstats import zonal_stats
import folium
import folium.raster_layers
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import branca.colormap as cm

### Preprocessing

In [ ]:
# File Name: Bangladesh_Latest_-_Global_Administrative_Boundaries.zip
# Description: Administrative boundaries, level 0, 1, 2, 3, and 4
# Source: World Food Programme (WFP)
# Format: geoJSON

BGD_border_file = r"C:\UN Ahead of the storm Hackathon\Adm_borders\Bangladesh_Latest_-_Global_Administrative_Boundaries\adm0.geojson"

In [ ]:
# Flood Hazard 2 Years - SSP1 Lower bound
# Source: https://giri.unepgrid.ch/map?list=explore
# Id MX-V95CQ-75NRI-9BEHJ
# Release date: Sun Apr 30 2023; Data value starting date: Sat Dec 31 2016; Data value ending date: Thu Dec 30 2100

flood_risk_file = r"C:\UN Ahead of the storm Hackathon\Flood_hurricane_weather\global_rcp26_h2glob.tif"

In [3]:

def crop_raster_to_polygon(raster_path, shapefile_path, output_path, crop=True):
    """
    Crop raster to polygon boundary using rasterio.mask
    
    Parameters:
    - raster_path: path to input raster
    - shapefile_path: path to polygon shapefile
    - output_path: path for cropped raster output
    - crop: if True, crop to polygon bounds; if False, just mask
    """
    
    # Read the shapefile
    shapes_gdf = gpd.read_file(shapefile_path)
    
    # Read the raster
    with rasterio.open(raster_path) as src:
        # Check if CRS match, reproject if needed
        if shapes_gdf.crs != src.crs:
            print(f"Reprojecting shapefile from {shapes_gdf.crs} to {src.crs}")
            shapes_gdf = shapes_gdf.to_crs(src.crs)
        
        # Extract geometries
        shapes = shapes_gdf.geometry.values
        
        # Crop the raster
        out_image, out_transform = rasterio.mask.mask(
            src, 
            shapes, 
            crop=crop,  # Crop to polygon bounds
            nodata=src.nodata
        )
        
        # Update metadata
        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
    
    # Write the cropped raster
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(out_image)
    
    print(f"Raster cropped and saved to: {output_path}")
    return output_path


In [ ]:
# Cropped global Flood Hazard 2 Years - SSP1 Lower bound file to Bangladesh borders
flood_risk_output_file = r"C:\UN Ahead of the storm Hackathon\Flood_hurricane_weather\BGD_cropped_global_rcp26_h2glob.tif"
flood_risk_BGD = crop_raster_to_polygon(flood_risk_file, BGD_border_file, flood_risk_output_file)

Raster cropped and saved to: C:\UN Ahead of the storm Hackathon\Flood_hurricane_weather\BGD_cropped_global_rcp26_h2glob.tif


In [ ]:
# File Name: hotosm_bgd_health_facilities_points_geojson.geojson
# Description: Points marking the location of health infrastructure in Bangladesh from OSM, including hospitals and clinics. Valuable for evaluating disaster impacts on essential services and emergency response capacity.
# Source: https://data.humdata.org/dataset/hotosm_bgd_health_facilities
# Format: GeoJSON


# Clean the HOTOSM hospital file from pharmacies and dentist
file_path = r'C:\UN Ahead of the storm Hackathon\Facilities\hotosm_bgd_health_facilities_points_geojson.geojson'
health_gdf = gpd.read_file(file_path)
hospitals_doctors = health_gdf[health_gdf['amenity'].isin(['hospital', 'birthing_center', 'doctors', 'clinic'])]
hospitals_doctors.to_file(r'C:\UN Ahead of the storm Hackathon\Facilities\hotosm_bgd_health_facilities_points_cleaned.geojson')

In [ ]:
# File Name: hotosm_bgd_education_facilities_points_geojson.geojson
# Description: Geolocated point dataset of educational facilities (schools, colleges, etc.) from OSM in Bangladesh. Useful for identifying critical public infrastructure and assessing their disaster exposure or accessibility.
# Source: https://data.humdata.org/dataset/hotosm_bgd_education_facilities
# Format: GeoJSON

# Clean the HOTOSM school file 
file_path_sch = r'C:\UN Ahead of the storm Hackathon\Facilities\hotosm_bgd_education_facilities_points_geojson.geojson'
school_gdf = gpd.read_file(file_path_sch)
school_gdf.info()
school_gdf['amenity'].unique()
schools = school_gdf[school_gdf['amenity'].isin(['school', 'college', 'university', 'kindergarten'])]
schools.to_file(r'C:\UN Ahead of the storm Hackathon\Facilities\hotosm_bgd_education_facilities_points_cleaned.geojson')

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7798 entries, 0 to 7797
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   name              7222 non-null   object  
 1   name:en           443 non-null    object  
 2   amenity           7696 non-null   object  
 3   building          307 non-null    object  
 4   operator:type     127 non-null    object  
 5   capacity:persons  55 non-null     object  
 6   addr:full         4 non-null      object  
 7   addr:city         665 non-null    object  
 8   source            1351 non-null   object  
 9   name:bn           260 non-null    object  
 10  osm_id            7798 non-null   int64   
 11  osm_type          7798 non-null   object  
 12  geometry          7798 non-null   geometry
dtypes: geometry(1), int64(1), object(11)
memory usage: 792.1+ KB


array(['school', 'college', 'university', 'kindergarten', None,
       'marketplace'], dtype=object)

In [ ]:
# Merge Bangladesh district borders to poverty data from "POVERTY MAPS OF BANGLADESH 2016" December 2020, Dhaka. 
# https://bbs.portal.gov.bd/sites/default/files/files/bbs.portal.gov.bd/page/5695ab85_1403_483a_afb4_26dfd767df18/2021-02-22-16-57-c64fb3d272175e7efea0b02de6a23eaa.pdf

bgd_adm2_polygons = r"C:\UN Ahead of the storm Hackathon\Adm_borders\Bangladesh_Latest_-_Global_Administrative_Boundaries\adm2.geojson"
poverty_data = r"C:\UN Ahead of the storm Hackathon\soc-ec\HCR_povertyrate_district.xlsx"

bgd_adm2_gdf = gpd.read_file(bgd_adm2_polygons)
poverty_df = pd.read_excel(poverty_data)

# Merge the data based on a common column
bgd_adm2_poverty_gdf = bgd_adm2_gdf.merge(poverty_df,
                                          left_on='name', 
                                          right_on='District',
                                          how='left')
bgd_adm2_poverty_gdf.to_file(r"C:\UN Ahead of the storm Hackathon\Adm_borders\Bangladesh_Latest_-_Global_Administrative_Boundaries\adm2_pov.geojson")
# The missing values from merging issues were inputted manually in QGIS

## Risk analysis

#### Identify flood risk zones

In [11]:
def raster_to_classified_polygons(raster_path, output_path, method='quantile'):
    """
    Classify raster into 3 groups and convert to vector polygons
    
    Parameters:
    - raster_path: path to input raster file
    - output_path: path for output shapefile
    - method: 'quantile', 'equal_interval', or 'manual'
    """
    
    # Read the raster
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)  # Read first band
        transform = src.transform
        crs = src.crs
        nodata = src.nodata
    
    # Mask nodata values
    if nodata is not None:
        valid_mask = raster_data != nodata
        valid_data = raster_data[valid_mask]
    else:
        valid_data = raster_data.flatten()
        valid_mask = np.ones_like(raster_data, dtype=bool)
    
    # Create classification based on method
    if method == 'quantile':
        # Use 33rd and 67th percentiles
        thresholds = np.percentile(valid_data, [33.33, 66.67])
    elif method == 'equal_interval':
        # Equal intervals between min and max
        min_val, max_val = np.min(valid_data), np.max(valid_data)
        interval = (max_val - min_val) / 3
        thresholds = [min_val + interval, min_val + 2*interval]
    else:
        # Manual thresholds - adjust these values as needed
        thresholds = [np.percentile(valid_data, 33), np.percentile(valid_data, 67)]
    
    # Create classified raster
    classified = np.zeros_like(raster_data, dtype=np.int32)
    
    # Assign classes: 1 (low), 2 (medium), 3 (high)
    classified[raster_data <= thresholds[0]] = 1
    classified[(raster_data > thresholds[0]) & (raster_data <= thresholds[1])] = 2
    classified[raster_data > thresholds[1]] = 3
    
    # Set nodata areas to 0
    if nodata is not None:
        classified[~valid_mask] = 0
    
    # Convert to polygons
    polygons = []
    values = []
    
    # Extract shapes for each class
    for class_val in [1, 2, 3]:
        mask = classified == class_val
        for geom, val in shapes(mask.astype(np.uint8), transform=transform):
            if val == 1:  # Only keep polygons where mask is True
                polygons.append(shape(geom))
                values.append(class_val)
    
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame({
        'class': values,
        'geometry': polygons
    }, crs=crs)
    
    # Add descriptive labels
    class_labels = {1: 'Low', 2: 'Medium', 3: 'High'}
    gdf['label'] = gdf['class'].map(class_labels)
    
    # Save to shapefile
    gdf.to_file(output_path)
    
    print(f"Classification thresholds: {thresholds}")
    print(f"Number of polygons created: {len(gdf)}")
    print(f"Polygons saved to: {output_path}")
    
    return gdf

In [ ]:
# Getting risk zones for foolds based on the Flood Hazard 2 Years - SSP1 Lower bound estimates
raster_file = flood_risk_BGD
output_file = r"C:\UN Ahead of the storm Hackathon\Risk files\flood_risk_zones.shp"

gdf = raster_to_classified_polygons(raster_file, output_file, method='quantile')

# Print summary statistics
print("\nClass distribution:")
print(gdf['class'].value_counts().sort_index())

Classification thresholds: [115. 274.]
Number of polygons created: 343698
Polygons saved to: C:\UN Ahead of the storm Hackathon\Risk files\flood_risk_zones.shp

Class distribution:
class
1    167449
2    134308
3     41941
Name: count, dtype: int64


In [ ]:
risk_zones_flood_gdf = gpd.read_file(r'C:\UN Ahead of the storm Hackathon\Risk files\flood_risk_zones.shp')

#### Identify poverty rates tresholds

In [ ]:
bgd_adm2_poverty_gdf_cleaned = gpd.read_file(r'C:\UN Ahead of the storm Hackathon\soc-ec\adm2_pov.geojson')

In [9]:
bgd_adm2_poverty_gdf_cleaned['HCR Upper(%)']
min_value =bgd_adm2_poverty_gdf_cleaned['HCR Upper(%)'].min()
max_value = bgd_adm2_poverty_gdf_cleaned['HCR Upper(%)'].max()
print(f'max value{max_value}')
print(f'min_value{min_value}')

max value70.8
min_value2.6


The treshold identified and original plan was to correct the risk assessments for other types of risks based on them, however, time was not enough, but this can be done in future.


- below 10% - do not change risk class
- 10 - 50% - up risk assessment 1 level
- over 50% - up  risk assessment 2 levels or leave high risk


### Making risk zones maps for schooling and medial services

**Workflow for creating isodistance lines from hospitals and schools:**

Ideally:
1) The projection should be changed projected cooordinate system to measure distances in meters
2) Network analysis in QGIS/ArcGIS/PostGIS should be used.

We worked in QGIS.
Due to lack of time to, mostly, work through the OSM line features to connect all the line features for the analysis and to additional operations, quick workaround is used:

1) Draw buffer zones in 0.05 degree (about 5.5 km) and 0.1 degree (about 11 km) around the hospitals (Vector -> Geoprocessing tools -> Buffer)
2) Dissolve each layer, so overlapping buffers merge (Vector -> Geoprocessing tools -> Dissolve)
3) Make difference polygons between buffer 0.1 degree and buffer 0.05 degree
4) Add buffer 0.05 degree polygons and the difference polygons into one shapefile (Medical_assistance_risk.shp).
4) Make difference polygons between Bangladesh state border and Medical_assistance_risk.shp polygons.
5) Add the difference polygons into the Medical_assistance_risk.shp
6) Take the Intersection between the Bangladesh state border and Medical_assistance_risk.shp polygons to get rid of the areas outside the state border.

For the schools the processing is the same except the risk zones are narrower: 0.01 degree (~1 km) and 0.05 (~5 km) degree.

The resulting zoning is used for mining data below.

## Using the risk assesment to mine data

### Calculate number of children affected by an event

In [14]:
# Files with risk classifications after QGIS step:
schooling_risk_gdf = gpd.read_file(r"C:\UN Ahead of the storm Hackathon\Risk files\Schools_attendance_risk_cr.shp")
medical_risk_gdp = gpd.read_file(r"C:\UN Ahead of the storm Hackathon\Risk files\Medical_assistance_risk_cr.shp")
# # Other files for risk assessments (repeating from above)
bgd_adm2_poverty_gdf_cleaned = gpd.read_file(r'C:\UN Ahead of the storm Hackathon\soc-ec\adm2_pov.geojson')
risk_zones_flood_gdf = gpd.read_file(r'C:\UN Ahead of the storm Hackathon\Risk files\flood_risk_zones.shp')

In [4]:
# From WorldPop Bangladesh age structures:
# https://hub.worldpop.org/geodata/summary?id=50363

# Files with children population
population_dir = os.path.join('.', 'Population')
tif_files_pop = [f for f in os.listdir(population_dir) if f.endswith('.tif')]
tif_files_pop

# ['bgd_f_0_2020_constrained_UNadj.tif', # girls under 1 year old
#  'bgd_f_10_2020_constrained_UNadj.tif', # girls ages 10-15
#  'bgd_f_15_2020_constrained_UNadj.tif', # same naming convention 
#  'bgd_f_1_2020_constrained_UNadj.tif',
#  'bgd_f_5_2020_constrained_UNadj.tif',
#  'BGD_kids_10-15_total.tif', # preprocessed by merging boys and girls data
#  'BGD_kids_5-10_total.tif', # preprocessed by merging boys and girls data
#  'BGD_kids_under_5_total.tif', # preprocessed by merging boys and girls data
#  'bgd_m_0_2020_constrained_UNadj.tif', # boys under 1 year old
#  'bgd_m_10_2020_constrained_UNadj.tif', # boys ages 10-15
#  'bgd_m_15_2020_constrained_UNadj.tif', # same naming convention 
#  'bgd_m_1_2020_constrained_UNadj.tif',
#  'bgd_m_5_2020_constrained_UNadj.tif']

['bgd_f_0_2020_constrained_UNadj.tif',
 'bgd_f_10_2020_constrained_UNadj.tif',
 'bgd_f_15_2020_constrained_UNadj.tif',
 'bgd_f_1_2020_constrained_UNadj.tif',
 'bgd_f_5_2020_constrained_UNadj.tif',
 'BGD_kids_10-15_total.tif',
 'BGD_kids_5-10_total.tif',
 'BGD_kids_under_5_total.tif',
 'bgd_m_0_2020_constrained_UNadj.tif',
 'bgd_m_10_2020_constrained_UNadj.tif',
 'bgd_m_15_2020_constrained_UNadj.tif',
 'bgd_m_1_2020_constrained_UNadj.tif',
 'bgd_m_5_2020_constrained_UNadj.tif']

### Calculate number of kids that can lose schooling in case of catastrophic event overall in Bangladesh

In [ ]:
# Calculate number of children that can lose schooling in case of catastrophic event overall in Bangladesh

selected_file = 'BGD_kids_5-10_total.tif'
file_path = os.path.join(population_dir, selected_file)
stats_sums = zonal_stats(schooling_risk_gdf, file_path, stats = ['sum'])
for i, (class_name, stat_sum) in enumerate(zip(schooling_risk_gdf['Label'], stats_sums)):
    print(f"Children between 5 and 10 in {class_name} risk of losing schooling days: {stat_sum['sum']:,.0f}")

Children between 5 and 10 in Low risk of losing schooling days: 3,766,931
Children between 5 and 10 in Medium risk of losing schooling days: 5,893,504
Children between 5 and 10 in High risk of losing schooling days: 4,580,140


### Calculate how many kids under 5 that live in high flood risk zones also are high risk of not getting medical assistance in time

In [61]:
### Calculate how many children under 5 that live in high flood risk zones also are high risk of not getting medical assistance in time

## From the raster with number of children under 5 per grid cell select those that are in high risk of flooding

high_risk_flood_areas = risk_zones_flood_gdf[risk_zones_flood_gdf['class'] == 3]
selected_file = 'BGD_kids_under_5_total.tif'
input_raster = os.path.join(population_dir, selected_file)
with rasterio.open(input_raster) as src:
    # Extract geometries for masking
    geometries = high_risk_flood_areas.geometry.values
    
    # Mask raster with Class 3 polygons
    masked_data, transform = mask(src, geometries, crop=True, nodata=src.nodata)
    
    # Update metadata for output
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": masked_data.shape[1],
        "width": masked_data.shape[2],
        "transform": transform,
        "nodata": src.nodata
    })
    
    # Save masked raster
    with rasterio.open('high_risk_flood_kids_under_5.tif', 'w', **out_meta) as dest:
        dest.write(masked_data)

In [15]:
## Calculate number of those children also in high medical risk areas

selected_file = 'high_risk_flood_kids_under_5.tif'
# file_path = os.path.join(population_dir, selected_file)
stats_sums = zonal_stats(medical_risk_gdp, selected_file, stats = ['sum'])
for i, (class_name, stat_sum) in enumerate(zip(medical_risk_gdp['Label'], stats_sums)):
    print(f"Children under 5 living in high flood risk zones also in {class_name} risk of not getting timely medical services: {stat_sum['sum']:,.0f}")

Children under 5 living in high flood risk zones also in Low risk of not getting timely medical services: 388,422
Children under 5 living in high flood risk zones also in Medium risk of not getting timely medical services: 343,367
Children under 5 living in high flood risk zones also in High risk of not getting timely medical services: 141,325


### Calculate how many kids under 5 that live in high poverty areas  also are high flood risk zones

In [ ]:
### Calculate how many children under 5 that live in high poverty areas also are in high flood risk zones

## From the raster with number of children under 5 per grid cell select those that are in high poverty areas

high_povety_areas = bgd_adm2_poverty_gdf_cleaned[bgd_adm2_poverty_gdf_cleaned['HCR Upper(%)'] > 50]
selected_file = 'BGD_kids_under_5_total.tif'
input_raster = os.path.join(population_dir, selected_file)
with rasterio.open(input_raster) as src:
    # Extract geometries for masking
    geometries = high_povety_areas.geometry.values
    
    # Mask raster with Class 3 polygons
    masked_data, transform = mask(src, geometries, crop=True, nodata=src.nodata)
    
    # Update metadata for output
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": masked_data.shape[1],
        "width": masked_data.shape[2],
        "transform": transform,
        "nodata": src.nodata
    })
    
    # Save masked raster
    with rasterio.open('high_povety_areas_kids_under_5.tif', 'w', **out_meta) as dest:
        dest.write(masked_data)

In [ ]:
## Calculate number of those children also in high flood risk areas

selected_file = 'high_povety_areas_kids_under_5.tif'
stats_sums = zonal_stats(high_risk_flood_areas, selected_file, stats=['sum'])

# Create a DataFrame for easier grouping
df = pd.DataFrame({
    'label': high_risk_flood_areas['label'],
    'count': [stat['sum'] if stat['sum'] is not None else 0 for stat in stats_sums]
})

# Group by label and sum
summary = df.groupby('label')['count'].sum()

print("Summary by flood risk level:")
for risk_level, total_count in summary.items():
    print(f"{risk_level} risk: {total_count:,.0f} children")

print(f"\nTotal children under 5 in all flood risk areas: {summary.sum():,.0f}")

Summary by flood risk level:
High risk: 82,093 children

Total children in all flood risk areas: 82,093


### Analysing risks along the high impact zone on the Ampan track

In [ ]:
# Creating the risk assessment files for only the impact area

risk_files = [
    'Schools_attendance_risk_cr.shp',
    'Medical_assistance_risk_cr.shp',
    'flood_risk_zones.shp',
    'adm2_poverty.shp'
]

clip_geom = r'C:\UN Ahead of the storm Hackathon\Flood_hurricane_weather\amphan_high_impact_zone.shp'
f_folder = r"C:\UN Ahead of the storm Hackathon\Risk files"
output_folder = r"C:\UN Ahead of the storm Hackathon\Risk files\Clipped_outputs"  # Optional: specify output folder

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Read clip boundary once (more efficient)
clip_boundary = gpd.read_file(clip_geom)

for f in risk_files:
    try:
        # Read the shapefile
        f_path = os.path.join(f_folder, f)
        polygons = gpd.read_file(f_path)
        
        print(f"Processing: {f}")
        
        # Ensure both have the same CRS
        if polygons.crs != clip_boundary.crs:
            clip_boundary_proj = clip_boundary.to_crs(polygons.crs)
        else:
            clip_boundary_proj = clip_boundary
        
        # Clip the polygons
        clipped = gpd.clip(polygons, clip_boundary_proj)
        
        # Create output filename and path
        output_name = f.replace('.shp', '_cropped.shp')
        output_path = os.path.join(output_folder, output_name)
        
        # Save the result
        clipped.to_file(output_path)
        
        print(f"Saved: {output_name} ({len(clipped)} features)")
        
    except Exception as e:
        print(f"Error processing {f}: {str(e)}")

Processing: Schools_attendance_risk_cr.shp
Saved: Schools_attendance_risk_cr_cropped.shp (3 features)
Processing: Medical_assistance_risk_cr.shp
Saved: Medical_assistance_risk_cr_cropped.shp (3 features)
Processing: flood_risk_zones.shp
Saved: flood_risk_zones_cropped.shp (16168 features)
Processing: adm2_poverty.shp
Saved: adm2_poverty_cropped.shp (10 features)


In [7]:
schooling_risk_am_gdf = gpd.read_file(r"C:\UN Ahead of the storm Hackathon\Risk files\Clipped_outputs\Schools_attendance_risk_cr_cropped.shp")


### Calculate number of kids with schooling at risk due to Amphan impact

In [10]:
# Calculate number of children with schooling at risk
files_pop = [
    'BGD_kids_10-15_total.tif',
    'BGD_kids_5-10_total.tif',
]

for f_pop in files_pop:

    file_path = os.path.join(population_dir, f_pop)
    stats_sums = zonal_stats(schooling_risk_am_gdf, file_path, stats = ['sum'])
    for i, (class_name, stat_sum) in enumerate(zip(schooling_risk_am_gdf['Label'], stats_sums)):
        print(f"Children {f_pop} in {class_name} risk of losing schooling days: {stat_sum['sum']:,.0f}")

Children BGD_kids_10-15_total.tif in Low risk of losing schooling days: 141,339
Children BGD_kids_10-15_total.tif in Medium risk of losing schooling days: 271,154
Children BGD_kids_10-15_total.tif in High risk of losing schooling days: 153,239
Children BGD_kids_5-10_total.tif in Low risk of losing schooling days: 142,453
Children BGD_kids_5-10_total.tif in Medium risk of losing schooling days: 274,928
Children BGD_kids_5-10_total.tif in High risk of losing schooling days: 153,521


### Calculate number of kids under 5 in high poverty areas and at risk of loosing medical services due to Amphan

In [13]:

# Read poverty data
poverty_am_gdf = gpd.read_file(r"C:\UN Ahead of the storm Hackathon\Risk files\Clipped_outputs\adm2_poverty_cropped.shp")

# Filter high poverty areas (fixed typo: povety -> poverty)
high_poverty_areas_am = poverty_am_gdf[poverty_am_gdf['HCR Upper('] > 50]

# Check if any high poverty areas exist
if len(high_poverty_areas_am) == 0:
    print("No areas found with poverty rate > 50%")
else:
    print(f"Found {len(high_poverty_areas_am)} high poverty areas")

    selected_file = 'BGD_kids_under_5_total.tif'
    input_raster = os.path.join(population_dir, selected_file)
    
    # Check if input raster exists
    if not os.path.exists(input_raster):
        print(f"Error: Raster file not found: {input_raster}")
    else:
        try:
            with rasterio.open(input_raster) as src:
                # Ensure CRS alignment
                if high_poverty_areas_am.crs != src.crs:
                    high_poverty_areas_am = high_poverty_areas_am.to_crs(src.crs)
                
                # Extract geometries for masking
                geometries = high_poverty_areas_am.geometry.values
                
                # Mask raster with high poverty polygons
                masked_data, transform = mask(src, geometries, crop=True, nodata=src.nodata)
                
                # Update metadata for output
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": masked_data.shape[1],
                    "width": masked_data.shape[2],
                    "transform": transform,
                    "nodata": src.nodata
                })
                
                # Create output path
                output_path = r"C:\UN Ahead of the storm Hackathon\Outputs\high_poverty_areas_kids_under_5_am.tif"
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                
                # Save masked raster
                with rasterio.open(output_path, 'w', **out_meta) as dest:
                    dest.write(masked_data)
                
                print(f"Successfully saved masked raster: {output_path}")
                
        except Exception as e:
            print(f"Error processing raster: {str(e)}")

Found 1 high poverty areas
Successfully saved masked raster: C:\UN Ahead of the storm Hackathon\Outputs\high_poverty_areas_kids_under_5_am.tif


In [16]:
## Calculate number of those children also in high medical risk areas
medical_risk_gdf_am = gpd.read_file(r'C:\UN Ahead of the storm Hackathon\Risk files\Clipped_outputs\Medical_assistance_risk_cr_cropped.shp')
selected_file = r'C:\UN Ahead of the storm Hackathon\Outputs\high_poverty_areas_kids_under_5_am.tif'
stats_sums = zonal_stats(medical_risk_gdf_am, selected_file, stats = ['sum'])
for i, (class_name, stat_sum) in enumerate(zip(medical_risk_gdf_am['Label'], stats_sums)):
    print(f"Children under 5 living in high poverty areas also in {class_name} risk of not getting timely medical services due to Amphan: {stat_sum['sum']:,.0f}")

Children under 5 living in high poverty areas also in Low risk of not getting timely medical services due to Amphan: 43,199
Children under 5 living in high poverty areas also in Medium risk of not getting timely medical services due to Amphan: 43,400
Children under 5 living in high poverty areas also in High risk of not getting timely medical services due to Amphan: 9,651
